# <center><b>Hello guys!<br>
   

## <center>This notebook was created to make it easier for you to understand Mobile Phones Market Data dataset and its character.
        

<center><img src="https://i.cdn.newsbytesapp.com/images/l142_5951592045581.jpg" width=700></img></center> 

## <center>I hope it will be useful and interesting for you!
          

##   <center>Let's move on!

# <center><b> Preparing data

In [ ]:
import pandas as pd, numpy as np
import matplotlib.pyplot as plt, seaborn as sns, plotly.express as px, plotly.figure_factory as ff
import scipy
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

In [ ]:
data = pd.read_csv('../input/ukrainian-market-mobile-phones-data/phones_data.csv')

In [ ]:
data = data.drop(columns='Unnamed: 0')

In [ ]:
data.head()

In [ ]:
data.shape

# <b><center>Missing data

Let's take a look at missing data.

In [ ]:
data.isna().sum()

Seems like we have work to do :)

## <b>battery_size

In [ ]:
data[data.battery_size.isna()]

<br>As we can see, battery data is missing for two iPhone model's with different characteristics and for 4 Android model's.

Let's create a method which will find 5 most similar phones for the phones with missing data. This way we will understand, which characteristics do have the similar phones.<br><br>

In [ ]:
class SimilarPhones():
    
    def __init__(self, rec_data):
        self.data_ = rec_data.copy(True)
        self.numeric_cols = data.select_dtypes(include=['int16', 'int32', 'int64', 'float16', 'float32', 'float64']).columns
        
    def get_numeric_indeces(self, num_cols, missing_cols):
        result = []
        for col in num_cols:
            if col in missing_cols:
                continue
            result.append(list(self.data_.columns.values).index(col))
        return result
        
    def normalize_data(self):
        for col in self.numeric_cols:
            max_d = self.data_[col].max()
            min_d = self.data_[col].min()
            self.data_[col] = (self.data_[col] - min_d)/(max_d - min_d)
        return self.data_
        
    def get_best(self, phone_idx, missing_cols = [], phones_amount=5,):
        distances = []
        phone = self.data_.iloc[phone_idx]
        ind = self.get_numeric_indeces(self.numeric_cols, missing_cols)
        res_data = self.data_.drop(columns=[])
        for p in res_data.values:
            dist = 0
            for col in range(len(res_data.columns)):
                if not col in ind:
                    continue
                dist = dist + np.absolute(float(p[col]) - float(phone[col]))
            distances.append(dist)
        res_data['distance'] = distances
        res_data = res_data.sort_values('distance')
        return res_data[1:phones_amount+1].index
        

<br>Let's create an object of our class.<br><br>

In [ ]:
finder = SimilarPhones(data)

<br>Let's take a look at the same phones for our phones with missing battery_size data.<br><br>

In [ ]:
b_idxs = data[data.battery_size.isna()].index
finder.normalize_data()

<br>Let's take a look at battery_sizes of the phones which are similar to ours.<br><br>

In [ ]:
for idx in b_idxs:
    name = data.iloc[idx].model_name
    result = data.iloc[finder.get_best(idx, ['battery_size'], 4)]
    result = result.battery_size.values
    print("Phone : {}, missing data from similar phones : {}".format(name,  result))
    

<br>We can see that several similar phones could have the same battery_size. We will simply choose the mode of each of this lists.<br><br>

In [ ]:
b_size = []
for idx in b_idxs:
    result = data.iloc[finder.get_best(idx, ['battery_size'], 4)].battery_size.values
    b_size.append(scipy.stats.mode(result)[0][0])
b_size

<br>Let's fill missing data with these values.<br><br>

In [ ]:
temp = data[data.battery_size.isna()]
temp.battery_size = b_size
data[data.battery_size.isna()] = temp

<br><br>

In [ ]:
data.isna().sum()

<br>No we don't have any missing data for battery_size column. Let's move on.<br><br>

## <b> os

In [ ]:
data[data.os.isna()]

<br>As we can't predict OS correctly (if we do, wrong predictions will seriously affect our data), we will fill missing values with 'Other'.<br><br>

In [ ]:
temp = data[data.os.isna()]
temp.os = temp.os.fillna('Other')
data[data.os.isna()] = temp

<br>Check if everything worked fine.<br><br>

In [ ]:
data.isna().sum()

<br>Everything is ok, let's move on.

### <br>lowest_price and highest_price

As we can't predict lowest_price and highest_price with only best_price, we will guess, that these values are missing simply fill it with best_price values

In [ ]:
temp = data[data.lowest_price.isna()]
temp.lowest_price = temp.best_price
temp.highest_price = temp.best_price
data[data.lowest_price.isna()] = temp

<br>Check if everything worked.<br><br>

In [ ]:
data.isna().sum()

## <b> screen_size

<br>Let's use our SimilarPhones class to fill missing data in screen_size and memory_size columns. We will also choose the mode as we did before.<br><br>

In [ ]:
data[data.screen_size.isna()]

In [ ]:
s_idxs = data[data.screen_size.isna()].index
s_size = []

for idx in s_idxs:
    name = data.iloc[idx].model_name
    result = data.iloc[finder.get_best(idx, ['screen_size'], 4)].screen_size.values
    print("Phone : {}, missing data from similar phones : {}".format(name,  result))
    s_size.append(scipy.stats.mode(result)[0][0])

In [ ]:
temp = data[data.screen_size.isna()]
temp.screen_size = s_size
data[data.screen_size.isna()] = temp

In [ ]:
data.isna().sum()

In [ ]:
data[data.memory_size.isna()]

In [ ]:
m_idxs = data[data.memory_size.isna()].index

for idx in m_idxs:
    name = data.iloc[idx].model_name
    result = data.iloc[finder.get_best(idx, ['memory_size'], 4)].memory_size.values
    print("Phone : {}, missing data from similar phones : {}".format(name,  result))

In [ ]:
data[data.memory_size.isna()].best_price.max()

<br>As we can see, maximum price is 1558 hrivnyas, which is'nt really much, and a smartphone might cost much more. These phones might be all push-button. Moreover, they are similar to each other, as there are lots of nan values below.
That means, that they can't have lots of memory. As the most popular variant of memory size from output below is 0.032 GB, we will fill missing data with this value.<br><br>

In [ ]:
data = data.fillna(0.032)

<br>Check if it worked.<br><br>

In [ ]:
data.isna().sum()

<br>We have dealt with missing data, let's move on.<br><br>

# <center> <b>EDA and visualization

## <b>Prices<br>

In [ ]:
px.histogram(x=data.best_price, labels={'x': 'Best price'})

In [ ]:
px.histogram(x=data.lowest_price, labels={'x': 'Lowest price'})

In [ ]:
px.histogram(x=data.highest_price, labels={'x': 'Highest price'})

### <br>Well, what can be seen from these plots:
&nbsp;&nbsp;&nbsp;&nbsp;• highest_price and best_price distributions looks pretty similar;<br>
&nbsp;&nbsp;&nbsp;&nbsp;• lowest_price's distribution differs a little bit from highest_price and best_price distributions, it could probably be so, because this data also &nbsp;&nbsp;&nbsp;&nbsp;contains used phones;<br>
&nbsp;&nbsp;&nbsp;&nbsp;• such similarity can also be caused by the way we filled missing data for these columns;<br>
&nbsp;&nbsp;&nbsp;&nbsp;• most of the phones have the price under 10000 hrivnyas;

## <b>Characteristics<br>

In [ ]:
px.histogram(x=data.screen_size, color_discrete_sequence=['lightgreen'], labels={'x': 'Screen size'})

In [ ]:
px.histogram(x=data.battery_size, color_discrete_sequence=['lightgreen'], labels={'x': 'Battery size'})

In [ ]:
memory_data = data.copy(True)
memory_data = memory_data.groupby('memory_size').count()
memory_data['memory'] = memory_data.index.astype('str')

In [ ]:
plt.figure(figsize=(19, 8))
plt.bar(memory_data.memory, memory_data.release_date, color='lightgreen')
plt.ylabel("Amount of phones")
plt.xlabel("Memory size")
plt.show()

In [ ]:
d_data = data.copy(True)
d_data = d_data.groupby('release_date').count()
d_data['rel'] = d_data.index
rel = d_data.index
new_rel = []
for d in rel:
    if len(d) < 7:
        temp = '0'+d
    else:
        temp = d
    new_rel.append(float(temp.split('-')[1]+'.'+temp.split('-')[0]))
d_data['rel'] = new_rel
d_data = d_data.sort_values('rel')

In [ ]:
px.bar(x = d_data.index, y = d_data.brand_name, color_discrete_sequence=['lightgreen'], labels={'x': 'Month', 'y': 'Phones amount'})

### <br>What can be seen from these plots:
&nbsp;&nbsp;&nbsp;&nbsp;• screen_size varies mostly between 5 and 7 inches, there are also many phone with the size of screen 1.4-3 inches, which could be &nbsp;&nbsp;&nbsp;&nbsp;press-button phones;<br>
&nbsp;&nbsp;&nbsp;&nbsp;• 3k, 4k, and 5k mAh are the most popular sizes of the phone's batteries;<br>
&nbsp;&nbsp;&nbsp;&nbsp;• most phones have its memory_size in the range of 16-256 GB, where 64 and 128 are the most popular memory_size;<br>
&nbsp;&nbsp;&nbsp;&nbsp;• many phones also have 0.032 memory_size, which can be caused by previous missing data filling with this value;<br>
&nbsp;&nbsp;&nbsp;&nbsp;• most phones have its memory_size in the range of 16-256 GB, where 64 and 128 are the most popular memory_size;<br>
&nbsp;&nbsp;&nbsp;&nbsp;• many phones also have 0.032 memory_size, which can be caused by previous missing data filling with this value;<br>
&nbsp;&nbsp;&nbsp;&nbsp;• more phones are being released as the time goes;<br>
&nbsp;&nbsp;&nbsp;&nbsp;• october of 2020 had the highest amount of released phones.

## <br> <b>Popularity<br>

In [ ]:
s_data = data.copy(True)
s_data = s_data.groupby('os').count()
s_data = s_data.sort_values('brand_name', ascending=False)

In [ ]:
px.bar(x = s_data.index, y = s_data.brand_name, color_discrete_sequence=['orange'], 
       labels={'x': 'Operating System', 'y': 'Phones amount'})

In [ ]:
br_data = data.copy(True)
br_data = br_data.groupby('brand_name').count()
br_data = br_data.sort_values('model_name', ascending=False)

In [ ]:
px.bar(x = br_data.index, y = br_data.model_name, color_discrete_sequence=['orange'], 
       labels={'x': 'Brand', 'y': 'Phones amount'})

In [ ]:
p_data = data.copy(True)
p_data = p_data.sort_values('popularity', ascending=False)[:40]

In [ ]:
px.bar(x = p_data.model_name, y = p_data.popularity, color_discrete_sequence=['orange'], 
       labels={'x': 'Phone model', 'y': 'Popularity'})

### <br>What can be seen from these plots:
&nbsp;&nbsp;&nbsp;&nbsp;• the most of phones have Android OS;<br>
&nbsp;&nbsp;&nbsp;&nbsp;• the amount of phones whith OxygenOS, WindowsPhone, EMUI and KAIOS is extremely low, which can mean, they aren't popular at all;<br>
&nbsp;&nbsp;&nbsp;&nbsp;• Samsung, Xiaomi, Apple, Motorola and Sigma mobile are top-5 brands by number of phone models;<br>
&nbsp;&nbsp;&nbsp;&nbsp;• iPhones, Google Pixel, Xiaomi Redmi along with Xiaomi Mi, Samsung Galaxy and Poco phones are the most popular phones.<br>

# <center><b><br>Correlation analysis

In [ ]:
plt.subplots(figsize=(12, 8))
sns.heatmap(data.corr(), annot=True, square=True)
plt.show()

<br>Pearson correlation says us, that the prices corellates between themselves mostly ideal. 
<br>Let's also check Spearman correlation.<br><br>

In [ ]:
plt.subplots(figsize=(20, 8))
sns.heatmap(scipy.stats.spearmanr(data)[0], annot=True, square=True, xticklabels=data.columns, yticklabels=data.columns)
plt.show()

Well, Spearman correlation thinks that only greatly highest_price correlates with lowest_price.
<br>That could mean that dependence between these two columns might be stable.

# <b><center>Thank you for reading this notebook! 

# <center>Good luck!